# ML ENGINEERING

studenta: Flintoaca Cristina 

In [31]:
import json
import os
from typing import Union
from google.cloud import storage
import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

## 1. Configurare

In [32]:
project_id = 'ml-eng-4'
region = 'us-central1'
dataset = 'MobilityDB'


In [33]:
!gsutil ls -p {project_id}

gs://dataprep-staging-459a5b56-5093-4fb7-8a20-9233431c5a82/
gs://ml-eng-team4bucket/


In [34]:
# Selectez bucketul echipei
bucket = 'gs://ml-eng-team4bucket/'

In [35]:
# VertexAI & BigQuery

vertex_ai.init(project = project_id,
               location = region,
               staging_bucket = bucket)

# Initializez clientul 
bq_client = bigquery.Client(project=project_id)

## 2. Incarcarea datelor & modelului din Big Query

In [36]:
# Incarcarea tabelului

query = """
SELECT * FROM `ml-eng-4.MobilityDB.mortality_final_table` 
"""
df= bq_client.query(query).to_dataframe()

In [37]:
df.head()

,country,code,region,region_noHICgroup,incgroup2,incgroup3,incgroup4,fragile,survey,year,...,mortality_rate,avg_access_to_electricity_rural,avg_access_to_electricity_urban,avg_current_health_expenditure,avg_physicians_per_1000,avg_gdp_per_capita,avg_gni_per_capita,avg_adolescents_out_of_school,avg_literacy_rate_adult,row_num
0,Afghanistan,AFG,South Asia,South Asia,Developing economies,Low income,Low income,True,NRVA,2011,...,4.86,20.11,82.2,58.19,0.14,557.64,2085.49,80.27,18.16,96
1,Afghanistan,AFG,South Asia,South Asia,Developing economies,Low income,Low income,True,NRVA,2011,...,4.86,72.24,82.2,27.01,0.27,516.75,2085.49,80.27,18.16,10
2,Afghanistan,AFG,South Asia,South Asia,Developing economies,Low income,Low income,True,NRVA,2011,...,4.06,20.11,82.2,58.19,0.12,189.95,2085.49,80.27,18.16,45
3,Afghanistan,AFG,South Asia,South Asia,Developing economies,Low income,Low income,True,NRVA,2011,...,4.86,72.24,82.2,58.19,0.05,281.11,2085.49,80.27,31.45,55
4,Afghanistan,AFG,South Asia,South Asia,Developing economies,Low income,Low income,True,NRVA,2011,...,4.86,20.11,82.2,58.19,0.12,281.11,2085.49,80.27,18.16,72


In [38]:
# Incarcarea modelului

model_id = 'ml-eng-4.MobilityDB.DNN_flintoaca_cristina'

model = bq_client.get_model(model_id)

print(model)

Model(reference=ModelReference(project_id='ml-eng-4', dataset_id='MobilityDB', model_id='DNN_flintoaca_cristina'))


##### Decrierea modelului (variabila target, variabile independente, ce am vrut sa prezic si cum)

Variabila target: rata mortalitatii

Variabilele independente: media vitaminei A acoperite, media accesului la electricitate in zonele rurale si urbane, media cheltuielilor pentru sanatate, numarul mediu de medici / 1000 locuritori, PIB-ul mediu / cap de locuitor, PIB-ul national brut mediu/ cap locuitor, media adolescentilor care termina scoala, media de alfabetizare a adultilor

Obiectivul: Prezicerea ratei mortalitatii in functie de variabilele independente alese. Am ales sa folosesc un model de DNN pentru a avea mai multa precizie si pentru ca un DNN regressor este capabil sa identifice relatiile complexe dintre variabilele independente si variabila target ceea ce ar putea duce, ulterior, la un model (poate) mai performant.  

## 3. Evaluarea modelului

In [39]:
eval_model_query = """
SELECT *
FROM
  ML.EVALUATE(MODEL`MobilityDB.DNN_flintoaca_cristina`)
  """
eval_df = bq_client.query(eval_model_query).to_dataframe()

In [40]:
eval_df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,3.09823,21.672886,0.176789,2.027053,0.652747,0.666548


##### Performanta modelului
R-squared are valoarea 0.65 ceea ce inseamna aproximativ 65% din varianta ratei mortalitatii este explicata de variabilele independente introduse in model

## 4. EXPLAIN.PREDICT

In [41]:
project_id = 'ml-eng-4'
dataset = 'MobilityDB'
model = 'ml-eng-4.MobilityDB.DNN_flintoaca_cristina'
table = 'ml-eng-4.MobilityDB.mortality_final_table'

In [42]:
explain_pred_query = f'''
SELECT *
FROM
  ML.PREDICT(MODEL `MobilityDB.DNN_flintoaca_cristina`, (
    SELECT
      avg_vitamin_a_coverage,
      avg_access_to_electricity_rural,
      avg_access_to_electricity_urban,
      avg_current_health_expenditure,
      avg_physicians_per_1000,
      avg_gdp_per_capita,
      avg_gni_per_capita,
      avg_adolescents_out_of_school,
      avg_literacy_rate_adult,
      mortality_rate
    FROM
      `MobilityDB.mortality_final_table`
    WHERE
      mortality_rate IS NOT NULL 
  )
);
'''

explanation = run_bq_query(explain_pred_query)

Finished job_id: c3f84d2e-447d-4895-8373-319b5df38071


In [43]:
explanation.head()

,predicted_mortality_rate,avg_vitamin_a_coverage,avg_access_to_electricity_rural,avg_access_to_electricity_urban,avg_current_health_expenditure,avg_physicians_per_1000,avg_gdp_per_capita,avg_gni_per_capita,avg_adolescents_out_of_school,avg_literacy_rate_adult,mortality_rate
0,6.732921,78.0,20.11,82.2,58.19,0.14,557.64,2085.49,80.27,18.16,4.86
1,6.732921,78.0,72.24,82.2,27.01,0.27,516.75,2085.49,80.27,18.16,4.86
2,6.732921,78.0,20.11,82.2,58.19,0.12,189.95,2085.49,80.27,18.16,4.06
3,6.732921,78.0,72.24,82.2,58.19,0.05,281.11,2085.49,80.27,31.45,4.86
4,6.732921,78.0,20.11,82.2,58.19,0.12,281.11,2085.49,80.27,18.16,4.86


## 5. Comparatie intre modele

In [57]:
models = {
    "LR_Nohai_Stefan_Lucian": "regression",
    "RF_baghiut_ioana": "regression",
    "DNN_flintoaca_cristina": "regression",
    "XGBoost_purcaras_andreea": "classification",
}

def get_model_evaluation(model_name, model_type):
    if model_type == 'classification':
        metrics = "accuracy, precision, recall, f1_score"
    elif model_type == 'regression':
        metrics = "mean_absolute_error, mean_squared_error, r2_score"  

    try:
        query = f"""
            SELECT 
                '{model_name}' AS model_name, 
                {metrics}
            FROM 
                ML.EVALUATE(MODEL `{project_id}.{dataset}.{model_name}`)
        """
        return run_bq_query(query) 
    except Exception as e:
        print(f"Error evaluating model {model_name}: {str(e)}")
        return pd.DataFrame()

In [58]:
comparison = [get_model_evaluation(model, type) for model, type in models.items()]
comparison_results_df = pd.concat(comparison)


Finished job_id: 9397c212-7361-4900-8ea4-d7f53950165f
Finished job_id: d49d0d48-b17b-470f-a1c3-42c8c029d02b
Finished job_id: 105860ca-388e-4e82-b7a1-9d6de00bcaca
Finished job_id: 95fc978c-7451-48a9-8f29-f1f333552705


In [60]:
comparison_results_df

,model_name,mean_absolute_error,mean_squared_error,r2_score,accuracy,precision,recall,f1_score
0,LR_Nohai_Stefan_Lucian,4.881218,48.110360,0.265280,NaN,NaN,NaN,NaN
0,RF_baghiut_ioana,2.572541,12.563697,0.804902,NaN,NaN,NaN,NaN
0,DNN_flintoaca_cristina,3.098230,21.672886,0.652747,NaN,NaN,NaN,NaN
0,XGBoost_purcaras_andreea,NaN,NaN,NaN,0.965938,0.9757,0.955299,0.965392


##### Care este cel mai bun model si de ce?
Din cele 4 modele, XGBoost este cel mai bun in ceea ce priveste performanta overall, intrucat scorul F1 de 0.96 indica ca modelul clasifica datele excelent. In cazul setului de date folosit, modelul de clasificare este mult mai eficient si mai performant decat modelele de regresie realizate.

In ceea ce priveste analizele de regresie, modelul Random Forest este cel mai eficient (r-squared = 0.80, mean_absolute_error = 3.57), intrucat reuseste sa explice aproximativ 80% din varianta ratei mortalitatii comparativ cu celelalte modele (de regresie logistica si deep neural networks).